In [1]:
# from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.kittiloader import *


import os
import sys
import time
import datetime
import argparse
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator
import time


image_folder = 'examples/'
config_path = 'config/v390.cfg'
weights_path = 'weights/v390_final.weights'
kitti_path = '/home/zijieguo/project/KITTI/'
class_path = 'data/coco.names'
conf_thres = 0.8
nms_thres = 0.4
batch_size = 1
n_cpu = 16
img_size = 416
use_cuda = True
CUDA_available = torch.cuda.is_available() and use_cuda

detections = torch.tensor([
    [  5.0433, 208.0689, 133.5634, 274.0513,   0.9992,   0.9995,   2.0000],
         [202.3678, 203.0705, 242.1279, 233.4327,   0.9980,   0.9999,   2.0000],
         [118.8502, 203.6419, 208.2199, 263.8680,   0.9891,   0.9758,   2.0000]
#          ,[271.3281, 201.2100, 280.0563, 207.2144,   0.9638,   0.9832,   2.0000],
#          [314.0755, 218.4319, 416.3620, 269.0359,   0.9204,   0.9999,   2.0000],
#          [293.7480, 207.2141, 319.6446, 226.7405,   0.8638,   0.9870,   2.0000],
#          [248.1488, 202.3865, 267.4640, 215.2374,   0.8588,   0.9984,   2.0000]
])


In [2]:
if detections[0] is not None:
        detections_with_distance = torch.zeros((detections.shape[0],detections.shape[1]+1))
        detections_with_distance[:,:-1] = detections
detections_with_distance
    


tensor([[  5.0433, 208.0689, 133.5634, 274.0513,   0.9992,   0.9995,   2.0000,
           0.0000],
        [202.3678, 203.0705, 242.1279, 233.4327,   0.9980,   0.9999,   2.0000,
           0.0000],
        [118.8502, 203.6419, 208.2199, 263.8680,   0.9891,   0.9758,   2.0000,
           0.0000]])

In [3]:
for detection in detections_with_distance:
    img_id = 8
    img_path = 'examples/000008.png'
    detection = detection.numpy()
    img_size_after_resize = img_size
    lidar_path = '%straining/velodyne/%06d.bin' % (kitti_path, img_id)
    calib = calibread('%straining/calib/%06d.txt' % (kitti_path, img_id))
    img = cv2.imread('/home/zijieguo/project/PyTorch-YOLOv3/examples/%06d.png' % img_id, cv2.IMREAD_UNCHANGED)
    # img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img_width_orig = img.shape[1]

    img_height_orig = img.shape[0]

    pad_x = max(img_height_orig - img_width_orig, 0) * (img_size_after_resize / max(img_width_orig, img_height_orig))
    pad_y = max(img_width_orig - img_height_orig, 0) * (img_size_after_resize / max(img_width_orig, img_height_orig))
    # Image height and width after padding is removed
    unpad_h = img_size_after_resize - pad_y
    unpad_w = img_size_after_resize - pad_x
    box_h = ((detection[3] - detection[1]) / unpad_h) * img_height_orig
    box_w = ((detection[2] - detection[0]) / unpad_w) * img_width_orig
    v_upper = ((detection[1] - pad_y // 2) / unpad_h) * img_height_orig
    u_left = ((detection[0] - pad_x // 2) / unpad_w) * img_width_orig
    v_bottom = v_upper + box_h
    u_right = u_left + box_w

    point_cloud = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 4)
    # orig_point_cloud = point_cloud # nx4

    # detections with shape: (x1, y1, x2, y2, object_conf, class_score, class_pred)


    P2 = calib["P2"] # 3x4 matris projection matrix after rectification
    # （u,v,1） = dot(P2, (x,y,z,1))
    Height_of_camera = 1.65
    fu = P2[0][0]  # for horizontal position
    fv = P2[1][1]
    # theta_c = np.pi/2
    D_rough = Height_of_camera * fv / (v_bottom - img_height_orig/2)
    # D_rough = Height_of_camera * (np.tan(theta_c + np.arctan((img_height_orig/2 - d_p)/fv)) - np.tan(theta_c - np.arctan(img_height_orig/(2*fv))))
    print(D_rough)
    if D_rough > 0:
        # remove points that are located behind the camera:
        point_cloud = point_cloud[point_cloud[:, 0] > (D_rough - 5), :]
        print(point_cloud.shape)
        
        # remove points that are located too far away from the camera:
        point_cloud = point_cloud[point_cloud[:, 0] < min(80, D_rough + 5), :]
        print(point_cloud.shape)

        point_cloud = point_cloud[point_cloud[:,2] > -Height_of_camera,:]
        print(point_cloud.shape)


6.0191195494967635
(52649, 4)
(44653, 4)
(34047, 4)
15.557979818717886
(8538, 4)
(6167, 4)
(5328, 4)
7.112372944615878
(45379, 4)
(38000, 4)
(27765, 4)


In [4]:
print(point_cloud)

[[ 9.647  3.235  0.553  0.41 ]
 [ 9.676  3.279  0.555  0.41 ]
 [ 9.654  3.305  0.554  0.66 ]
 ...
 [ 2.46   2.517 -1.525  0.04 ]
 [ 2.467  2.533 -1.533  0.17 ]
 [ 2.541  2.623 -1.587  0.14 ]]


In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

axes_limits = [
    [-10, 80], # X axis range
    [-30, 30], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']

def draw_point_cloud(ax, title, point_cloud, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """
#         ax.scatter(*np.transpose(velo_frame[:, axes]), s=point_size, c=velo_frame[:, 3], cmap='cool')
        ax.scatter(point_cloud[:,axes[0]],point_cloud[:,axes[1]], s=1, c=point_cloud[:, 3], facecolor='black', cmap='gray')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        if len(axes) > 2:
            ax.set_xlim3d(*axes_limits[axes[0]])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
        else:
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)

                       
f, ax3 = plt.subplots(3, 1, figsize=(15, 25))
draw_point_cloud(
    ax3[0], 
    'Velodyne scan, XZ projection (Y = 0), the car is moving in direction left to right',
    point_cloud,
    axes=[0, 2] # X and Z axes
)
draw_point_cloud(
    ax3[1], 
    'Velodyne scan, XY projection (Z = 0), the car is moving in direction left to right', 
    point_cloud,
    axes=[0, 1] # X and Y axes
)
draw_point_cloud(
    ax3[2], 
    'Velodyne scan, YZ projection (X = 0), the car is moving towards the graph plane', 
    point_cloud,
    axes=[1, 2] # Y and Z axes
)
plt.show()

KeyboardInterrupt: 

In [6]:



#         Tr_velo_to_cam_orig = calib["Tr_velo_to_cam"]
#         R0_rect_orig = calib["R0_rect"] # 3x3

#         R0_rect = np.eye(4)
#         R0_rect[0:3, 0:3] = R0_rect_orig # 3x3 -> 4x4 up left corner
#         ########################################################################
#         # R0_rect: example
#         # array([[ 0.99, 0.01, 0.01,   0 ],
#         #        [ 0.01, 0.99, 0.01,   0 ],
#         #        [ 0.01, 0.01, 0.99,   0 ],
#         #        [    0,    0,    0,   1 ]])
#         ########################################################################

#         Tr_velo_to_cam = np.eye(4)
#         Tr_velo_to_cam[0:3, :] = Tr_velo_to_cam_orig # 3x4 -> 4x4 up left corner
#         ########################################################################
#         # Tr_velo_to_cam:
#         # Tr_velo_to_cam = [ R_velo_to_cam,    t_velo_to_cam ]
#         #                  [             0,                1 ]
#         # Rotation matrix velo -> camera 3x3, translation vector velo ->camera 1x3
#         ########################################################################

#         point_cloud_xyz = point_cloud[:, 0:3] # num_point x 3 (x,y,z,reflectance) reflectance don't need
#         point_cloud_xyz_hom = np.ones((point_cloud.shape[0], 4))
#         point_cloud_xyz_hom[:, 0:3] = point_cloud[:, 0:3] # (point_cloud_xyz_hom has shape (num_points, 4))
#         # the 4th column are all 1

#         # project the points onto the image plane (homogeneous coords):
#         img_points_hom = np.dot(P2, np.dot(R0_rect, np.dot(Tr_velo_to_cam, point_cloud_xyz_hom.T))).T # (point_cloud_xyz_hom.T has shape (4, num_points))
#         # (U,V,_) = P2 * R0_rect * Tr_velo_to_cam * point_cloud_xyz_hom
#         # normalize: (U,V,1)
#         img_points = np.zeros((img_points_hom.shape[0], 2))
#         img_points[:, 0] = img_points_hom[:, 0]/img_points_hom[:, 2]
#         img_points[:, 1] = img_points_hom[:, 1]/img_points_hom[:, 2]

#         # transform the points into (rectified) camera coordinates:
#         point_cloud_xyz_camera_hom = np.dot(R0_rect, np.dot(Tr_velo_to_cam, point_cloud_xyz_hom.T)).T # (point_cloud_xyz_hom.T has shape (4, num_points))
#         # normalize: (x,y,z,1)
#         point_cloud_xyz_camera = np.zeros((point_cloud_xyz_camera_hom.shape[0], 3))
#         point_cloud_xyz_camera[:, 0] = point_cloud_xyz_camera_hom[:, 0]/point_cloud_xyz_camera_hom[:, 3]
#         point_cloud_xyz_camera[:, 1] = point_cloud_xyz_camera_hom[:, 1]/point_cloud_xyz_camera_hom[:, 3]
#         point_cloud_xyz_camera[:, 2] = point_cloud_xyz_camera_hom[:, 2]/point_cloud_xyz_camera_hom[:, 3]

#         point_cloud_camera = point_cloud
#         point_cloud_camera[:, 0:3] = point_cloud_xyz_camera # reserve reflection

#         ########################################################################
#         # point_cloud               n x 4   original xyzr value before cali in velo coordinate
#         # point_cloud_xyz           n x 3   xyz value before cali in velo coordinate
#         # point_cloud_xyz_hom       n x 4   xyz1 in velo coordinate
#         # point_cloud_xyz_camera    n x 4   xyz1 in camera coordinate
#         # point_cloud_camera        n x 4   xyzr in camera coordinate
#         # img_points_hom            n x 3   uv_
#         # img_points                n x 2   UV
#         ########################################################################


#         row_mask = np.logical_and(
#                         np.logical_and(img_points[:, 0] >= u_left,
#                                        img_points[:, 0] <= u_right),
#                         np.logical_and(img_points[:, 1] >= v_upper,
#                                        img_points[:, 1] <= v_bottom))

#         # filter out point are not in frustum area
#         frustum_point_cloud_xyz = point_cloud_xyz[row_mask, :] # (needed only for visualization)
#         frustum_point_cloud = point_cloud[row_mask, :]
#         frustum_point_cloud_xyz_camera = point_cloud_xyz_camera[row_mask, :]
#         frustum_point_cloud_camera = point_cloud_camera[row_mask, :]

#         # randomly sample 512 points in the frustum point cloud:


#         if frustum_point_cloud.shape[0] == 0:
#              detection[7] = D_rough
#              return torch.tensor(detection)
#         # elif frustum_point_cloud.shape[0] < 512:
#         #     row_idx = np.random.choice(frustum_point_cloud.shape[0], 512, replace=True)
#         # else:
#         #     row_idx = np.random.choice(frustum_point_cloud.shape[0], 512, replace=False)

#         frustum_point_cloud_xyz_camera = frustum_point_cloud_xyz_camera[row_idx, :]
#         ransac = linear_model.RANSACRegressor()
#         ransac.fit(frustum_point_cloud_xyz_camera[:,1].reshape(-1,1),frustum_point_cloud_xyz_camera[:,0].reshape(-1,1))

#         right_side_distance = ransac.predict([[frustum_point_cloud_xyz_camera[:,1].max()]])[0][0]
#         left_side_distance = ransac.predict([[frustum_point_cloud_xyz_camera[:,1].min()]])[0][0]

#         detection[7] = min(min(left_side_distance,right_side_distance),D_rough-2)
#         print('image id:', img_id)
#         print('Rough estimation %d, \n ransac estimation: %d %d, \n final estimation: %d' %(rough_D,left_side_distance,right_side_distance,detection[7]))
#         return torch.tensor(detection)

#     else:# might be a problem
#         detection[7] = float('nan')
#         return torch.tensor(detection)